# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [268]:
# Libraries

import pandas as pd
import numpy as np
import random

# Challenge 1 - Index Functions

In this challenge, we will practice the advanced functions introduced in the lesson. However, before we start, we will load and evaluate our dataset. 

Load the dataset from Ironhack's database:
* db: `admissions predict`
* table: `admissions`

In [293]:
# We import the dataset admission predict

admissions = pd.read_csv('/Users/GiadaSartori/Documents/IRONHACK/Lab/Week2/lab-advanced-pandas/your-code/data/Admission_Predict.csv')
display(admissions)

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,316,104,3,3.0,3.5,8.00,1,0.72
2,3,322,110,3,3.5,2.5,8.67,1,0.80
3,4,314,103,2,2.0,3.0,8.21,0,0.65
4,5,330,115,5,4.5,3.0,9.34,1,0.90
...,...,...,...,...,...,...,...,...,...
380,381,324,110,3,3.5,3.5,9.04,1,0.82
381,382,325,107,3,3.0,3.5,9.11,1,0.84
382,383,330,116,4,5.0,4.5,9.45,1,0.91
383,384,312,103,3,3.5,4.0,8.78,0,0.67


In [294]:
# We use the function head() in order to display the first top 5 elements of the dataframe admissions

admissions.head()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,316,104,3,3.0,3.5,8.00,1,0.72
2,3,322,110,3,3.5,2.5,8.67,1,0.80
3,4,314,103,2,2.0,3.0,8.21,0,0.65
4,5,330,115,5,4.5,3.0,9.34,1,0.90


In [295]:
# We want to understand the shape of the dataframe admissions

admissions.shape

(385, 9)

Remove trailing spaces at the end of the column names if there are any.


In [296]:
# We use str.rstrip() function on the respective column name/s to strip the trailing space in pandas

# We create a function which will remove extra tailing whitespace from the data
# We pass dataframe as a parameter here

def whitespace_remover(admissions):
   
    # iterating over the columns
    for i in admissions.columns:
         
        # checking datatype of each columns
        if admissions[i].dtype == 'object':
             
            # applying strip function on column
            admissions[i] = admissions[i].map(str.strip)
        else:
             
            # if condn. is False then it will do nothing.
            pass
 
# applying whitespace_remover function on dataframe
whitespace_remover(admissions)

# printing dataframe
admissions


,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,316,104,3,3.0,3.5,8.00,1,0.72
2,3,322,110,3,3.5,2.5,8.67,1,0.80
3,4,314,103,2,2.0,3.0,8.21,0,0.65
4,5,330,115,5,4.5,3.0,9.34,1,0.90
...,...,...,...,...,...,...,...,...,...
380,381,324,110,3,3.5,3.5,9.04,1,0.82
381,382,325,107,3,3.0,3.5,9.11,1,0.84
382,383,330,116,4,5.0,4.5,9.45,1,0.91
383,384,312,103,3,3.5,4.0,8.78,0,0.67


Let's evaluate the dataset by looking at the `head` function.

In [297]:
# your code here

admissions.head()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,316,104,3,3.0,3.5,8.00,1,0.72
2,3,322,110,3,3.5,2.5,8.67,1,0.80
3,4,314,103,2,2.0,3.0,8.21,0,0.65
4,5,330,115,5,4.5,3.0,9.34,1,0.90


Before beginning to work with this dataset and evaluating graduate admissions data, we will verify that there is no missing data in the dataset. Do this in the cell below.

In [298]:
# your code here

#In order to check for the missing data in the dataframe admission_predict, we can use:
#- admission_predict.isnull(), which returns a boolean mask (True or False) of null values
#- admission_predict.isnull().sum(), which returns a series with the names of the columns where the null values, which are summed, then appear

admissions.isnull()



,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...
380,False,False,False,False,False,False,False,False,False
381,False,False,False,False,False,False,False,False,False
382,False,False,False,False,False,False,False,False,False
383,False,False,False,False,False,False,False,False,False


In [299]:
admissions.isnull().sum()

#We realize that there is no missing data in our dataframe admission_predict. 

Serial No.           0
GRE Score            0
TOEFL Score          0
University Rating    0
SOP                  0
LOR                  0
CGPA                 0
Research             0
Chance of Admit      0
dtype: int64

Interestingly, there is a column that uniquely identifies the applicants. This column is the serial number column. Instead of having our own index, we should make this column our index. Do this in the cell below. Keep the column in the dataframe in addition to making it an index.

In [300]:
#Option 1: we use the set_index() method to make a column index

admissions.set_index('Serial No.', inplace=True)

#We return again our dataframe admission_predict

admissions

,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
Serial No.,,,,,,,,
1,337,118,4,4.5,4.5,9.65,1,0.92
2,316,104,3,3.0,3.5,8.00,1,0.72
3,322,110,3,3.5,2.5,8.67,1,0.80
4,314,103,2,2.0,3.0,8.21,0,0.65
5,330,115,5,4.5,3.0,9.34,1,0.90
...,...,...,...,...,...,...,...,...
381,324,110,3,3.5,3.5,9.04,1,0.82
382,325,107,3,3.0,3.5,9.11,1,0.84
383,330,116,4,5.0,4.5,9.45,1,0.91


In [277]:
#Option 2: we can read our csv file again with pandas, and specify that the index column will be Serial No, which is the column in the first position of our dataframe

#admission_predict = pd.read_csv('/Users/GiadaSartori/Documents/IRONHACK/Lab/Week2/lab-advanced-pandas/your-code/data/Admission_Predict.csv', index_col=0)
#display(admission_predict)

Turns out that `GRE Score` and `CGPA` also uniquely identify the data. Show this in the cell below.

In [301]:
# your code here

#We sum, per each row in the dataframe, the GRE score and CGPA values, and we create a new column called 'Combined GRE CGPA' which represents the summed value

admissions['Combined_GRE_CGPA'] = admissions['GRE Score'].astype('str') + admissions['CGPA'].astype('str')
admissions['Combined_GRE_CGPA']


Serial No.
1      3379.65
2       3168.0
3      3228.67
4      3148.21
5      3309.34
        ...   
381    3249.04
382    3259.11
383    3309.45
384    3128.78
385    3339.66
Name: Combined_GRE_CGPA, Length: 385, dtype: object

In [302]:

len(admissions['Combined_GRE_CGPA'].unique())


385

Replace the index with an index comprised of two columns - `GRE Score` and `CGPA`. Remove the columns from the dataset as well. Make sure to use `inplace=True`.

In [303]:
# your code here

admissions.set_index(['GRE Score', 'CGPA'], inplace = True)

admissions

,,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit,Combined_GRE_CGPA
GRE Score,CGPA,,,,,,,
337,9.65,118,4,4.5,4.5,1,0.92,3379.65
316,8.00,104,3,3.0,3.5,1,0.72,3168.0
322,8.67,110,3,3.5,2.5,1,0.80,3228.67
314,8.21,103,2,2.0,3.0,0,0.65,3148.21
330,9.34,115,5,4.5,3.0,1,0.90,3309.34
...,...,...,...,...,...,...,...,...
324,9.04,110,3,3.5,3.5,1,0.82,3249.04
325,9.11,107,3,3.0,3.5,1,0.84,3259.11
330,9.45,116,4,5.0,4.5,1,0.91,3309.45


Now change the index back to a sequence starting at zero using the `reset_index` function. Make sure to use `inplace=True`.

In [304]:
# your code here

admissions.reset_index(inplace = True)
admissions

,GRE Score,CGPA,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit,Combined_GRE_CGPA
0,337,9.65,118,4,4.5,4.5,1,0.92,3379.65
1,316,8.00,104,3,3.0,3.5,1,0.72,3168.0
2,322,8.67,110,3,3.5,2.5,1,0.80,3228.67
3,314,8.21,103,2,2.0,3.0,0,0.65,3148.21
4,330,9.34,115,5,4.5,3.0,1,0.90,3309.34
...,...,...,...,...,...,...,...,...,...
380,324,9.04,110,3,3.5,3.5,1,0.82,3249.04
381,325,9.11,107,3,3.0,3.5,1,0.84,3259.11
382,330,9.45,116,4,5.0,4.5,1,0.91,3309.45
383,312,8.78,103,3,3.5,4.0,0,0.67,3128.78


# Challenge 2 - Advanced Functions

In this part of the lab, we would like to test complex conditions on the entire dataset at once. Let's start by finding the number of rows where the CGPA is higher than 9 and the student has conducted research.

In [305]:
# your code here

admissions

,GRE Score,CGPA,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit,Combined_GRE_CGPA
0,337,9.65,118,4,4.5,4.5,1,0.92,3379.65
1,316,8.00,104,3,3.0,3.5,1,0.72,3168.0
2,322,8.67,110,3,3.5,2.5,1,0.80,3228.67
3,314,8.21,103,2,2.0,3.0,0,0.65,3148.21
4,330,9.34,115,5,4.5,3.0,1,0.90,3309.34
...,...,...,...,...,...,...,...,...,...
380,324,9.04,110,3,3.5,3.5,1,0.82,3249.04
381,325,9.11,107,3,3.0,3.5,1,0.84,3259.11
382,330,9.45,116,4,5.0,4.5,1,0.91,3309.45
383,312,8.78,103,3,3.5,4.0,0,0.67,3128.78


Now return all the rows where the CGPA is greater than 9 and the SOP score is less than 3.5. Find the mean chance of admit for these applicants.

In [306]:
# your code here

change_admit = admissions[(admissions["CGPA"]>9) & (admissions["SOP"]<3.5)]
change_admit


,GRE Score,CGPA,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit,Combined_GRE_CGPA
28,338,9.40,118,4,3.0,4.5,1,0.91,3389.4
62,327,9.02,114,3,3.0,3.0,0,0.61,3279.02
140,326,9.11,114,3,3.0,3.0,1,0.83,3269.11
217,324,9.01,111,4,3.0,3.0,1,0.82,3249.01
381,325,9.11,107,3,3.0,3.5,1,0.84,3259.11


In [284]:
change_admit.mean()

/var/folders/vy/72w35wd12ms4_h7n214795k40000gn/T/ipykernel_17246/3393592334.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  change_admit.mean()


GRE Score            328.000
CGPA                   9.130
TOEFL Score          112.800
University Rating      3.400
SOP                    3.000
LOR                    3.400
Research               0.800
Chance of Admit        0.802
dtype: float64

We would like to create a deciding factor column for each student. We standardize several columns and then pick the most important factor from a lookup table. If the standardized value is above 0.8, the student will be accepted.

We will start by creating a standardized column for `CGPA`, `GRE Score` and `LOR`. We will name these columns `CGPA_std`, `GRE_std`, and `LOR_std` respecively.

Recall that standardizing a column is done by subtracting the mean of the column from all observations in the column and then dividing each observation in the column by the column's standard deviation.

In the cell below, write a standardization function (a function that takes a column as input and returns a standardized column as output). Make sure to use the `numpy` versions of mean and standard deviation.

In [285]:
def standardize(col):
    """
    This function takes a column from a dataframe 
    and returns a standardized column by subtracting the column's mean
    and dividing by the column's standard deviation.
    """
    
    # your code here --> In order to normalize a dataframe we use: normalized_df=(df-df.mean())/df.std()

    return (col - col.mean())/col.std()

Now create the standardized columns `CGPA_std`, `GRE_std`, and `LOR_std` and add them to the `admissions` dataframe.

In [307]:
#Standardization of CGPA and creation of a new column, CGPA_std, to add to dataframe admissions

admissions['CGPA_std'] = (admissions['CGPA'] - admissions['CGPA'].mean())/admissions['CGPA'].std()
pd.DataFrame(admissions['CGPA_std'])

,CGPA_std
0,1.747900
1,-0.991211
2,0.121034
3,-0.642597
4,1.233279
...,...
380,0.735259
381,0.851463
382,1.415886
383,0.303641


In [308]:
#Standardization of GRE Score and creation of a new column, GRE_std, to add to dataframe admissions

admissions['GRE_std'] = (admissions['GRE Score'] - admissions['GRE Score'].mean())/admissions['GRE Score'].std()
pd.DataFrame(admission_predict['GRE_std'])


,GRE_std
0,1.753381
1,-0.063367
2,0.455704
3,-0.236391
4,1.147798
...,...
380,0.628727
381,0.715239
382,1.147798
383,-0.409414


In [319]:
#Standardization of LOR and creation of a new column, LOR_std, to add to dataframe admissions

admissions['LOR_std'] = (admissions['LOR'] - admissions['LOR'].mean())/admissions['LOR'].std()


KeyError: 'LOR'

We will generate the decision choice at random using the code below. Please run the cell.

In [313]:
# Libraries
from random import choices

In [314]:
std_columns = ['CGPA_std', 'GRE_std', 'LOR_std']

decision_choice = choices(std_columns, k=admissions.shape[0])

Now create the deciding column using the `lookup` function. The lookup column is `decision_choice` found above. Call the column resulting from the lookup function `deciding_column` and add it to the `admissions` dataframe.

In [312]:
# your code here

admissions['Deciding Column'] = admissions.lookup(admissions.index, decision_choice)
admissions

/var/folders/vy/72w35wd12ms4_h7n214795k40000gn/T/ipykernel_17246/4149607597.py:3: FutureWarning: The 'lookup' method is deprecated and will be removed in a future version. You can use DataFrame.melt and DataFrame.loc as a substitute.
  admissions['Deciding Column'] = admissions.lookup(admissions.index, decision_choice)


KeyError: 'LOR_std'

Create a column called `decision` in the `admissions` dataframe. Assign 1 to this column if the value of `deciding_column` is greater than 0.8 and 0 otherwise.

In [ ]:
# your code here

How many applicants will be accepted to the program using the decision column? Compute the result below.

In [ ]:
# your code here

# Challenge 3 - Method Chaining

To increase our coding efficiency, let's make a number of changes to our dataframe in one line of code.

In the cell below, remove all non character symbols from the column names, replace all spaces in column names to underscores, and change all upper case characters to lower case. Assign these new column values to `admissions.columns`.

In [ ]:
# your code here

In the cell below, give all student with a university rating of 4 or higher a 10 point boost on their GRE score and split the column into 4 bins using the `cut` function. Assign this new score to the variable `adjusted_gre`.

In [ ]:
# your code here